In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']

In [3]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list, low, high):
    features_to_gen = []
    for function in function_list:
        for i in range(low, high+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

features_to_gen = get_features_to_gen([fe.count, fe.unique_count, 
                                       fe.cumulative_count, 
                                       fe.reverse_cumulative_count, 
                                       fe.variance, fe.count_std_over_mean], 
                                      2, 2)# len(categorical_features))
# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count])
# features_to_gen = get_features_to_gen([fe.count])

features_to_gen

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DepTime'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'UniqueCarrier'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Origin'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Dest'],
  'function': <function automl_libs.feature_engineering.co

In [4]:
len(features_to_gen)

168

In [5]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/' 

In [6]:
from automl_app import logger_config

logger_config.config(project_path+'log_project1.log')

In [7]:
def gs_params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 80, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':10,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric': 'auc' 
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.005, # if == lr_init, then no lr decay
            'batch_size': np.random.choice([32, 64, 128, 256, 512, 1024, 2048]),
            "pred_batch_size": 100000,
            'max_ep': 3,
            'patience': 1,
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'dense_units': 100,
            'num_dense_n_layers': 1,
            'drop_rate': 0.2,
            'combined_dense_n_layers': 2,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max'  # MUST HAVE
        }
    return params, seed

In [8]:
gs_params_gen('nn')

({'batch_size': 64,
  'cat_emb_outdim': 50,
  'combined_dense_n_layers': 2,
  'dense_units': 100,
  'drop_rate': 0.2,
  'ep_for_lr': 1,
  'lr_fin': 0.005,
  'lr_init': 0.01,
  'max_ep': 3,
  'mode': 'max',
  'monitor': 'val_auc',
  'nn_seed': 15236012,
  'num_dense_n_layers': 1,
  'patience': 1,
  'pred_batch_size': 100000},
 15236005)

In [12]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, gs_params_gen)

2018-06-30 16:38:47 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #66  | Run record file [./output/last_run_record.json] found. Continue from the previous run...
2018-06-30 16:38:47 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #95  | generate todo list
2018-06-30 16:38:47 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #111 | STAGE: FEATURE_ENGINEERING
2018-06-30 16:38:47 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #115 | STAGE: VALIDATION_DOWNSAMPLING_GEN
2018-06-30 16:38:47 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #119 | STAGE: CONCAT_DATA
2018-06-30 16:38:47 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #123 | STAGE: GRID_SEARCH


[20]	cv_agg's auc: 0.713961 + 0.00148717
[40]	cv_agg's auc: 0.719743 + 0.00144944
[60]	cv_agg's auc: 0.725047 + 0.00144991


2018-06-30 16:39:09 | DEBUG | automl_libs.grid_search       | _lgb_gs    | #101 | [do_preds] is True, generating predictions ...
2018-06-30 16:39:09 | DEBUG | automl_libs.grid_search       | _lgb_gs    | #102 | Retrain model using best_round and all data...


[80]	cv_agg's auc: 0.730639 + 0.00133984
[16]	training's auc: 0.713792
[32]	training's auc: 0.719284
[48]	training's auc: 0.723836
[64]	training's auc: 0.728905
[80]	training's auc: 0.733624


2018-06-30 16:39:18 | WARNING | automl_libs.grid_search       | _lgb_gs    | #115 | DEBUG: roc of test: 0.7262977700853064
2018-06-30 16:39:18 | INFO  | automl_libs.grid_search       | _lgb_gs    | #121 | LGB predictions(1530391129) saved in ./output/gs_saved_preds/.
2018-06-30 16:39:18 | DEBUG | automl_libs.grid_search       | gs         | #52  | ./output/lgb_grid_search.csv updated
2018-06-30 16:39:18 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #127 | STAGE: STACKNET
2018-06-30 16:39:19 | DEBUG | automl_libs.stacknet          | layer1     | #34  | data_id: flight_data_ordinal  
	x_train: (700000, 176)	x_test: (100000, 176)
	y_train type: <class 'dict'>
	compatible_models: {'LGB', 'NN'}
 
2018-06-30 16:39:19 | DEBUG | automl_libs.base_layer_utils.BaseLayerResultsRepo| __init__   | #231 | load StackNet saves from file
2018-06-30 16:39:19 | DEBUG | automl_libs.stacknet          | layer1     | #59  | 1530326907 already processed in StackNet, so removed it from chosen_gs_result

roc of test 1530391159__LOGREG_layer2 0.7432737384756698
